# 탐색적 데이터 분석 - EDA

## 데이터 둘러보기

In [ ]:
import pandas as pd

In [ ]:
# 데이터 경로
data_path = '/kaggle/input/porto-seguro-safe-driver-prediction/'

train = pd.read_csv(data_path + 'train.csv', index_col='id')
test = pd.read_csv(data_path + 'test.csv', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

In [ ]:
train.shape, test.shape

train 보다 test 데이터가 더 많음

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
submission.head()

운전자가 보험금을 청구할 확률이 얼마나 되는지를 예측해야 함

In [ ]:
train.info()

데이터 타입은 int64 와 float64 로 구성

ind, reg, car, calc 로 분류

피처명에 다양한 메타 정보 포함

결측값이 없다고 나오지만 -1 이 결측값이므로 -1을 np.NaN 으로 변환한 다음 개수 파악 필요

In [ ]:
import numpy as np
import missingno as msno

In [ ]:
# 훈련 데이터 복사본에서 -1을 np.NaN 으로 변환
train_copy = train.copy().replace(-1, np.NaN)

# 결측값 기각화(처음 28개만)
msno.bar(df=train_copy.iloc[:, 1:29], figsize=(13, 6))

막대 그래프가 낮을수록 결측값이 많다는 뜻임

ps_reg_03, ps_car_02_cat, ps_car_05_cat 피처에 특히 결측값이 많음

In [ ]:
msno.bar(df=train_copy.iloc[:, 29:], figsize=(13, 6))

ps_car_14 에 결측값이 좀 있고 나머지 피처에는 거의 없음

### 피처 요약표

In [ ]:
def resumetable(df):
    print(f'데이터셋 형상: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['데이터 타입'])
    summary['결측값 개수'] = (df == -1).sum().values  # 피처별 -1 개수
    summary['고윳값 개수'] = df.nunique().values
    summary['데이터 종류'] = None
    for col in df.columns:
        if 'bin' in col or col == 'target':
            summary.loc[col, '데이터 종류'] = '이진형'
        elif 'cat' in col:
            summary.loc[col, '데이터 종류'] = '명목형'
        elif df[col].dtype == float:
            summary.loc[col, '데이터 종류'] = '연속형'
        elif df[col].dtype == int:
            summary.loc[col, '데이터 종류'] = '순서형'
            
    return summary

In [ ]:
summary = resumetable(train)
summary

In [ ]:
# 명목형 피처만 추출
summary[summary['데이터 종류'] == '명목형'].index

In [ ]:
# 실수형 피처만 추출
summary[summary['데이터 타입'] == 'float64'].index

## 데이터 시각화

데이터 시각화를 통해 모델링에 필요한 피처는 무엇이고, 필요 없는 피처는 무엇인지 선별

In [ ]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

### 타깃값 분포

In [ ]:
def write_percent(ax, total_size):
    '''도형 객체를 순회하며 막대 그래프 상단에 타깃값 비율 표시'''
    for patch in ax.patches:
        height = patch.get_height()  # 도형 높이(데이터 개수)
        width = patch.get_width()  # 도형 너비
        left_coord = patch.get_x()  # 도형 왼쪽 테두리의 x 축 위치
        percent = height / total_size * 100  # 타깃값 비율
        
        # (x, y) 좌표에 텍스트 입력
        ax.text(x=left_coord + width / 2.0, # x 축 위치
                y=height + total_size * 0.001,  # y 축 위치
                s='{:1.1f}%'.format(percent),  # 입력 텍스트
                ha='center',  # 가운데 저열ㄹ
               )

In [ ]:
mpl.rc('font', size=15)
plt.figure(figsize=(7, 6))

ax = sns.countplot(x='target', data=train)
write_percent(ax, len(train))
ax.set_title('Target Distribution')

전체 운전자 중 3.6% 만 보험금을 청구

차 사고가 그리 흔하게 나지 않음

소수의 운전자만 보험금을 청구했고 타깃값이 굉장히 불균형함을 알 수 있음

타깃값이 불균형하기 때문에 작은 타깃값 1을 잘 예측하는게 중요

> 타깃값의 비율 차이가 크면 비율이 작은 타깃값을 잘 예측하는 게 중요

> 각각의 고윳값마다 타깃값 비율이 다른 피처여야 모델링에 도움이 됨

> 통계적 유효성이 높아야(신뢰구간이 좁아야) 모델링에 도움이 됨

> 종합하면, 고윳값별 타깃값 1 비율이 충분히 차이가 나고 신뢰구간도 작은 피처여야 모델링에 도움이 됨. 그렇지 않은 피처는 제거하는 게 좋음

### 이진 피처

In [ ]:
import matplotlib.gridspec as gridspec

In [ ]:
def plot_target_ratio_by_features(df, features, num_rows, num_cols, size=(12, 18)):
    mpl.rc('font', size=9)
    plt.figure(figsize=size)  # 전체 Figure 크기 설정
    grid = gridspec.GridSpec(num_rows, num_cols)  # 서브플롯 배치
    plt.subplots_adjust(wspace=0.3, hspace=0.3)  # 서브플롯 좌우/상하 여백 설정
    
    for idx, feature in enumerate(features):
        ax = plt.subplot(grid[idx])
        # ax 축에 고윳값별 타깃값 1 비율을 막대 그래프로 그리기
        sns.barplot(x=feature, y='target', data=df, palette='Set2', ax=ax)

In [ ]:
bin_features = summary[summary['데이터 종류'] == '이진형'].index  # 이진 피처
# 이진 피처 고윳값별 타깃값 1 비율을 막대 그래프로 그리기
plot_target_ratio_by_features(train, bin_features, 6, 3)  # 6행 3열 배치

ps_ind_06_bin 피처를 보면 값이 0일 때 타깃 값이 1인 비율이 4%(0.04) 정도이고 나머지 96%는 타깃값 0임

값이 1일 때는 타깃값 1이 2.8% 정도 차지함

종합하면 ps_ind_06_bin 피처는 고윳값별로 타깃값 비율이 다름 -> 타깃값을 추정하는 예측력이 있음

신뢰구간도 좁기 때문에 모델링에 도움이 되는 피처임


- 모델링 시 제거할 피처

|서브플롯 위치|피처명|제거해야 하는 이유|
|---|---|---|
|(1행2열 ~ 2행2열)|ps_ind_10_bin ~ ps_ind_13_bin|신뢰구간이 넓어 통계적 유효성이 떨어짐|
|(4행0열 ~ 5행2열)|ps_calc_15_bin ~ ps_calc_20_bin|고윳값별 타깃값 비율 차이가 없어 타깃값 예측력이 없음|

### 명목형 피처

In [ ]:
nom_features = summary[summary['데이터 종류'] == '명목형'].index  # 명목형 피처

plot_target_ratio_by_features(train, nom_features, 7, 2)  # 7행 2열

1번 그래프의 **ps_ind_02_cat** 피처는 결측값 -1이 다른 고윳값들보다 타깃값 1 비율이 큼

신뢰구간이 넓다는 점을 감안해도 비율이 확실히 큼

이런 상황에서 결측값을 다른 값으로 대체하면 모델 성능이 더 나빠질 수 있음

결측값 자체가 타깃값에 대한 예측력이 있기 때문

결측값을 그대로 두고 모델링 진행 -> -1도 하나의 고윳값이라고 간주

**ps_car_02_cat** 피처는 -1일때 타깃값 1 비율은 0% 입니다. 이 경우 ps_car_02_cat 피처 값이 -1 이면 타깃값이 0이라고 판단해도 됨

결측값이 타깃값을 예측하는 데 도움을 줌

> 결측값 자체에 타깃값 예측력이 있다면 고윳값으로 간주(결측값 처리 필요 없음)

**ps_ind_02_cat, ps_ind_04_cat, ps_car_01_cat** 피처를 보자.

-1을 제외하고 나머지 고윳값은 타깃값 1 비율이 비슷하고 -1일 때의 신뢰구간이 상대적으로 넓다.

그렇더라도 -1의 신뢰하한과 다른 고윳값들의 신뢰상한 간 차이가 큼

따라서 -1의 신뢰구간이 넓다는 점을 감안해도 -1과 다른 고윳값들 간 타깃값 1 비율에 차이가 있음

즉, 고윳값 간 타깃값 1 비율에 차이가 있으므로 모델링에 필요한 피처임

**ps_car_10_cat** 피처를 보면 세 고윳값은 타깃값 1의 평균 비율이 비슷함

고윳값 2의 신뢰구간이 유독 넓음

이 피처를 제거해야 할까? 애매함

이런 경우 해당 피처를 제거한 경우와 제거하지 않은 경우에 성능이 어떻게 다른지 비교해보는 것도 좋은 방법임

테스트해본 결과 **ps_car_10_cat** 피처를 제거하지 않은 경우에 성능이 더 좋았음

> 명목형 피처는 모두 모델링에 이용

### 순서형 피처

In [ ]:
ord_features = summary[summary['데이터 종류'] == '순서형'].index  # 순서형 피처

plot_target_ratio_by_features(train, ord_features, 8, 2, (12, 20))  # 8행 2열

**ps_ind_14** 피처의 경우 고윳값 0, 1, 2, 3 의 타깃값 비율은 큰 차이가 없고 고윳값 4의 신뢰구간은 넓음

통계적 유효성이 떨어지므로 제거

**ps_calc_04 ~ ps_calc_14** 피처들은 모두 고윳값별 타깃값 비율이 거의 비슷함

타깃값 비율이 다른 고윳값도 있긴 하지만, 그 고윳값은 신뢰구간이 넓어 통계적 유효성이 떨어짐

이 피처들 모두 제거

- 제거해야 할 순서형 피처

|피처명|제거해야 하는 이유|
|---|---|
|ps_ind_14|타깃값 비율의 신뢰구간이 넓어 통계적 유효성이 떨어짐|
|ps_calc_04 ~ ps_calc_14|고윳값별 타깃값 비율 차이가 없음. 타깃값 비율이 다르더라도 신뢰구간이 넓어 통계적 유효성이 떨어짐|

### 연속형 피처

연속형 피처는 연속된 값이므로 고윳값이 굉장히 많음

> 고윳값이 아주 많을 때는 몇 개의 구간으로 나눠 구간별 비율을 알아볼 수 있음

In [ ]:
pd.cut([1.0, 1.5, 2.1, 2.7, 3.5, 4.0], 3)

이 방식은 연속형 데이터를 범주형 데이터로 바꾸는 효과가 있음

In [ ]:
cont_features = summary[summary['데이터 종류'] == '연속형'].index  # 연속형 피처

plt.figure(figsize=(12, 16))  # Figure 크기 설정
grid = gridspec.GridSpec(5, 2)  # GridSpec 객체 생성
plt.subplots_adjust(wspace=0.2, hspace=0.4)  # 서브플롯 간 여백 설정

for idx, cont_feature in enumerate(cont_features):
    # 값을 5개 구간으로 나누기
    train[cont_feature] = pd.cut(train[cont_feature], 5)
    
    ax = plt.subplot(grid[idx])  # 분포도를 그릴 서브플롯 설정
    sns.barplot(x=cont_feature, y='target', data=train, palette='Set2', ax=ax)
    ax.tick_params(axis='x', labelrotation=10)  # x축 라벨 회전

- 제거해야 할 연속형 피처

|피처명|제거해야 하는 이유|
|---|---|
|ps_calc_01 ~ ps_calc_03|구간별 타깃값 비율 차이가 없음|

나머지 피처는 고윳값별로 타깃값 비율이 서로 다름

타깃값 예측력이 있는 피처라고 볼 수 있음

### 연속형 피처 2

> 상관관계가 강한 피처들은 예측력도 비슷하므로 하나만 남겨두는 게 좋음

결측값이 있으면 상관관계를 올바르게 구하지 못하므로 먼저 결측값을 제거해야 함

In [ ]:
train_copy = train_copy.dropna()  # np.NaN 값 삭제

In [ ]:
plt.figure(figsize=(10, 8))
cont_corr = train_copy[cont_features].corr()  # 연속형 피처 간 상관관계
sns.heatmap(cont_corr, annot=True, cmap='OrRd')  # 히트맵 그리기

가장 강한 상관관계를 보이는 피처는 **ps_car_12** 와 **ps_car_14** 입니다.

두 피처 간 상관계수는 0.77이고 둘 중 하나를 제거해야 할 만큼 강한 상관관계를 보이는 건 아닙니다.

그렇지만 테스트 결과 **ps_car_14** 피처를 제거하니 성능이 더 좋아졌습니다.

**ps_reg_02** 와 **ps_reg_03** 피처 간 상관계수는 0.76으로 두 번째로 상관관계가 강함

테스트 결과 둘 중 하나를 제거하니 오히려 성능이 떨어졌음 -> 그대로 남겨둠